In [7]:
import openpyxl
import csv

In [8]:
def convert_to_csv(file_name):
    """Convert excel to csv."""
    wb = openpyxl.load_workbook(file_name, read_only = True)
    name_without_suffix = file_name.split(".")[0]
    print(wb.sheetnames)
    sh = wb["Raw Data"]
    print("creating excel file")
    with open(name_without_suffix + ".csv", 'w', newline="") as file_handle:
        csv_writer = csv.writer(file_handle)
        for row in sh.iter_rows(values_only = True):
            csv_writer.writerow([cell for cell in row])
    wb.close()


In [20]:
#!pip install pywin32
# xlsb conversion
# https://pypi.org/project/pywin32/
import win32com.client
import os
import time

In [23]:
def save_csv_file(work_sheet, file_name_list, filename):
    """Saving a worksheet to a csv file and add the csv file name to a list, naming convention applied
    Args:
        param work_sheet (win32com.client.CDispatch) worksheet to save csv 
        param file_name_list (list): list holding names 
    """

    work_sheet_name = work_sheet.name
    output_csv_name = os.getcwd() + "\\" + filename +'.csv'
    work_sheet.SaveAs(output_csv_name)
    file_name_list.append(output_csv_name)

    return file_name_list

def xl_file_to_csv(xl_file_file_path,filename,sheet_name = False):
    """Open a workbook, get the index of the worksheet (sh_index) and save sheet(s) as csv files.
    Args:
        xl_file_file_path: workbook path
        sheet_name: name of sheet
    """
    start_time = time.process_time()
    start_clock = time.perf_counter()

    xl_app = win32com.client.Dispatch("Excel.Application")
    xl_app.Visible = 0
    xl_app.DisplayAlerts = 0

    work_book = xl_app.Workbooks.Open(xl_file_file_path)
    csv_file_list = []
    worksheets = [sheet.Name for sheet in work_book.Worksheets]
    
    if isinstance(sheet_name,bool):
        for sheet in worksheets:
            work_sheet = sheet
            csv_file_list = save_csv_file(work_sheet, csv_file_list, filename)
    else:
        if sheet_name not in worksheets:
            work_book.Close(SaveChanges=0)
            xl_app.Quit()
            raise Exception("Target sheet not in the file")
            return

        work_sheet = work_book.Worksheets(sheet_name)
        csv_file_list = save_csv_file(work_sheet, csv_file_list, filename)

    work_book.Close(SaveChanges=0)
    xl_app.Quit()

    # Print runtime
    print('win32com process: {}'.format(time.process_time() - start_time))
    print('win32com counter: {}'.format(time.perf_counter() - start_clock))

    return csv_file_list

def main():
    for file in os.listdir():
        if file.endswith(".xlsb"):
            src_csv = xl_file_to_csv(
                xl_file_file_path = os.path.abspath(file),
                filename = file,
                sheet_name = 'Raw Data'
            )
            if src_csv is not None:
                print('')
                print('CSV file(s) created: ')
                print('-----------------------\n')
                print(*src_csv, sep="\n")